In [8]:
import pandas as pd
import re
file_path = 'Your_file.csv'
df = pd.read_csv(file_path)
particle_column = 'Nanomaterial'
cell_type_column = 'Cell type'

# Fill missing values in the Columns with empty strings
df[particle_column] = df[particle_column].fillna('')
df[cell_type_column] = df[cell_type_column].fillna('')

# Get user input for the type of particles
particle_input = input("Enter the type of particles to display (gold, silver, other): ").lower()

# Create separate data frames for gold, silver, and other particles
gold_df = df[df[particle_column].str.lower().str.contains('gold|au')]
silver_df = df[df[particle_column].str.lower().str.contains('silver')]
other_df = df[~(df[particle_column].str.lower().str.contains('gold|au|silver'))]

# Select the data frame based on user input
if particle_input == 'gold':
    particle_df = gold_df
    particle_type = 'Gold'
elif particle_input == 'silver':
    particle_df = silver_df
    particle_type = 'Silver'
else:
    particle_df = other_df
    particle_type = 'Other'

# Get user input for the cell type
cell_type_input = input("Enter the cell type (Hek, hela): ").lower()
filtered_df = particle_df[particle_df[cell_type_column].str.lower().str.contains(cell_type_input)].copy()

def extract_value_and_unit(value):
    match = re.match(r'([0-9.]+)\s*([a-zA-ZµM]+)', str(value))
    if match:
        return float(match.group(1)), match.group(2)
    return None, None

unique_nanomaterials = filtered_df[particle_column].unique()

print(f"\nCytotoxicity Analysis for {particle_type} particles on {cell_type_input} cells:")

for nanomaterial in unique_nanomaterials:
    nanomaterial_df = filtered_df[filtered_df[particle_column] == nanomaterial].copy()
    nanomaterial_df[['Value', 'Unit']] = nanomaterial_df['Cytotoxicity (mg/ml)'].apply(lambda x: pd.Series(extract_value_and_unit(x)))

    max_rows_nanomaterial = nanomaterial_df.sort_values('Value', ascending=False).groupby('Unit').head(2).drop_duplicates().copy()
    
    # Drop unwanted columns
    columns_to_drop = ['Reference', 'Unnamed: 7']
    max_rows_nanomaterial = max_rows_nanomaterial.drop(columns=columns_to_drop)
    
    # Check if there are any rows to display
    if not max_rows_nanomaterial.empty:
        print(f"\nCytotoxicity Analysis for {particle_type} particles ({nanomaterial}) on {cell_type_input} cells:")
        for _, row in max_rows_nanomaterial.iterrows():
            output_string = f"{nanomaterial} nanomaterial of size {row['Size (nm)']} with shape as {row['Shape']} and with {row['Capping Agent']} capping agent and toxicity {row['Cytotoxicity (mg/ml)']} has better cellular uptake"
            print(output_string)



Cytotoxicity Analysis for Gold particles on hela cells:

Cytotoxicity Analysis for Gold particles (Gold NP) on hela cells:
Gold NP nanomaterial of size 10-20nm with shape as Sphere and with Pongamia pinnata extract capping agent and toxicity 100 mg/ml has better cellular uptake
Gold NP nanomaterial of size 3–10 nm with shape as Sphere and with chitosan (CH) capping agent and toxicity 75 µM- 85% cell viability loss has better cellular uptake
Gold NP nanomaterial of size 110.1 ± 5.3 nm with shape as Sphere and with polyethylene glycol (PEG) capping agent and toxicity 25.37 µM has better cellular uptake
Gold NP nanomaterial of size 3.6 nm with shape as Sphere and with PEG capping agent and toxicity 25 nM has better cellular uptake

Cytotoxicity Analysis for Gold particles (Gold nm
) on hela cells:
Gold nm
 nanomaterial of size 40 nm with shape as ROD and with CTAB capping agent and toxicity 15 µg/mL has better cellular uptake
Gold nm
 nanomaterial of size 40 nm with shape as SPHERE and w